In [8]:
from subprocess import check_output

import numpy as np
from keras.preprocessing.image import ImageDataGenerator


In [9]:
def get_test_set_files_paths_arr_classes():
    fpa = []
    fpa.append(check_output(["ls", "seedlings_data/test/"]).decode("utf8").strip().split("\n"))
    filez = np.array(fpa)
    classes = np.array(check_output(["ls", "seedlings_data/train"]).decode("utf8").strip().split("\n"))
    return filez, classes

In [10]:
def get_predicted_classes(classes, predictions_vals): 
    max_indexes = np.argmax(predictions_vals, axis=1)
    pred_class = classes[max_indexes]
    pred_class = np.expand_dims(pred_class, axis=0)
    return pred_class


In [11]:
def get_subm_result(files_names_arr, classes_pred):
    conc = np.concatenate((files_names_arr, classes_pred), axis=0)
    conc = conc.astype('str')
    return np.apply_along_axis(lambda d: d[0] + ',' + d[1], 0, conc)

In [7]:
preds = np.load('seedl_subms/InceptionV3_preds_kfold10ep25_cat_cross_ent_0.98236.npy')
preds2 = np.load('seedl_subms/incep_preds_kfold10ep30_lr000026_cat_cross_ent0.97984.npy')
preds3 = np.load('seedl_subms/test_inc_rnd_crop_averaging_preds0.98362.npy')
preds4 = np.load('seedl_subms/resnet50_preds_kfold10ep40_lr8e-4_nodrop_b16_0.98110.npy')
preds5 = np.load('seedl_subms/resnet50_preds_kfold10ep40_lr8e-4_nodrop_b16_imgs_normalized0.97732.npy')
preds6 = np.load('seedl_subms/test_inc_rnd_crop_averaging_preds_drop0.5_0.97858.npy')
preds7 = np.load('seedl_subms/xcep_tst.npy')

# this resulted in 0.98488
# preds3 = np.load('seedl_subms/ResNet50_preds_kfold10ep25_cat_cross_ent_0.97481.npy') # 0.98110 + 0.98236 + 0.97984

predsx = (preds + preds2 + preds3 + preds4 + preds5 + preds6 + preds7) / 7
# predsx = np.reshape(np.hstack((preds, preds2, preds3, preds4, preds4)), (794, 12, 5))
# predsx = np.average(predsx, axis=2, weights=[0.98236, 0.97984, 0.97481, 0.98110, 0.97732])

filez, classes = get_test_set_files_paths_arr_classes()

predicted_classes = get_predicted_classes(classes, predsx)

res = get_subm_result(filez, predicted_classes)


res.tofile("seedl_subms/preds_avg_w_avg.csv", sep='\n')




[[  2.21705690e-05   4.33387651e-06   4.50923582e-06 ...,   1.73723522e-05
    8.83092052e-01   1.11145599e-05]
 [  2.06741827e-05   1.01800294e-05   1.09808328e-05 ...,   1.59849930e-05
    1.44443715e-05   4.19648842e-05]
 [  4.11588936e-06   9.90560079e-06   8.76891760e-05 ...,   2.12565309e-06
    1.84851333e-06   8.86754905e-01]
 ..., 
 [  1.12912492e-05   1.23567586e-05   2.35683571e-05 ...,   8.54473480e-06
    3.62146498e-06   8.31062099e-01]
 [  1.44396062e-05   9.01338106e-01   2.46805475e-06 ...,   1.87725255e-05
    2.14643473e-06   2.61164315e-06]
 [  2.88025854e-02   1.76553581e-05   3.53836188e-05 ...,   1.40235547e-05
    9.32185293e-06   1.45924099e-05]]


In [12]:
preds = np.load('seedl_subms/FINETUNE.npy')

filez, classes = get_test_set_files_paths_arr_classes()

predicted_classes = get_predicted_classes(classes, preds)

res = get_subm_result(filez, predicted_classes)


res.tofile('seedl_subms/FINETUNE.csv', sep='\n')


## Voting precictions:

In [91]:
preds = np.load('seedl_subms/InceptionV3_preds_kfold10ep25_cat_cross_ent_0.98236.npy')
preds2 = np.load('seedl_subms/incep_preds_kfold10ep30_lr000026_cat_cross_ent0.97984.npy')
preds3 = np.load('seedl_subms/test_inc_rnd_crop_averaging_preds0.98362.npy')
preds4 = np.load('seedl_subms/resnet50_preds_kfold10ep40_lr8e-4_nodrop_b16_0.98110.npy')
preds5 = np.load('seedl_subms/resnet50_preds_kfold10ep40_lr8e-4_nodrop_b16_imgs_normalized0.97732.npy')
preds6 = np.load('seedl_subms/test_inc_rnd_crop_averaging_preds_drop0.5_0.97858.npy')

# this resulted in 0.98488
# preds3 = np.load('seedl_subms/ResNet50_preds_kfold10ep25_cat_cross_ent_0.97481.npy') # 0.98110 + 0.98236 + 0.97984

def vote_pred(preds_list):
    vot_res = np.zeros((794, 12), dtype=np.int64)
    for predictions in preds_list:
        pred_max = np.argmax(predictions, axis=1)
        for idx in range(len(pred_max)):
            np.add.at(vot_res, (idx, pred_max[idx]), 1)
    return vot_res


v = vote_pred([preds, preds3, preds4])
v

TypeError: '>' not supported between instances of 'int' and 'str'